In [1]:
import sys
from joblib import load

sys.path.insert(0, '..')
sys.path.insert(0, '../..')
sys.path.insert(0, '../../..')
sys.path.insert(0, '../../../..')
sys.path.insert(0, '../../../../..')
sys.path.insert(0, '../../../../../..')

import torch
from torch import optim

from reimplemented_approaches.proactive_conformance_checking.models import LSTMCollectiveIDP
from reimplemented_approaches.proactive_conformance_checking.training import Training
from reimplemented_approaches.proactive_conformance_checking.data_prep_split_encode import PrefixDataset


In [2]:
# Load encoders:
# Load prepared and encoded datasets
train_set, val_set, test_set = PrefixDataset.load_datasets(save_path="../../../data_preparation/Helpdesk/collective/")

print(train_set.tensors[0].size())
print(train_set.tensors[1].size())
print(train_set.tensors[2].size())
print(train_set.tensors[3].size())
print(train_set.tensors[4].size())

encoders = load("../../../data_preparation/Helpdesk/collective/encoders.pkl")
print(encoders)

activity_ids = encoders.get('activity_ids') 
activity_ids_vocab_size_with_default = len(list(activity_ids.keys())) + 1 
print("Activities: ", activity_ids_vocab_size_with_default)

resource_ids = encoders.get('resource_ids')
resource_ids_vocab_size_with_default = len(list(resource_ids.keys())) + 1
print("Resources: ", resource_ids_vocab_size_with_default)

month_ids = encoders.get('month_ids') 
month_ids_vocab_size_with_default = len(list(month_ids.keys())) + 1

print("Months: ", month_ids_vocab_size_with_default)

number_trace_attr = train_set.tensors[3].size(1)
print("Number trace attributes: ", number_trace_attr)

number_deviations_y = len([d for d in encoders.get('deviations')])
print("Number deviatons y: ", number_deviations_y)


torch.Size([11386, 15])
torch.Size([11386, 15])
torch.Size([11386, 15])
torch.Size([11386, 0])
torch.Size([11386, 6])
{'activity_ids': {'Assign seriousness': 1, 'Closed': 2, 'Create SW anomaly': 3, 'DUPLICATE': 4, 'INVALID': 5, 'Insert ticket': 6, 'RESOLVED': 7, 'Require upgrade': 8, 'Resolve SW anomaly': 9, 'Resolve ticket': 10, 'Schedule intervention': 11, 'Take in charge ticket': 12, 'VERIFIED': 13, 'Wait': 14}, 'resource_ids': {'Value 1': 1, 'Value 10': 2, 'Value 11': 3, 'Value 12': 4, 'Value 13': 5, 'Value 14': 6, 'Value 15': 7, 'Value 16': 8, 'Value 17': 9, 'Value 18': 10, 'Value 19': 11, 'Value 2': 12, 'Value 20': 13, 'Value 21': 14, 'Value 22': 15, 'Value 3': 16, 'Value 4': 17, 'Value 5': 18, 'Value 6': 19, 'Value 7': 20, 'Value 8': 21, 'Value 9': 22}, 'month_ids': {'10_2010': 1, '10_2011': 2, '10_2012': 3, '10_2013': 4, '11_2010': 5, '11_2011': 6, '11_2012': 7, '11_2013': 8, '12_2010': 9, '12_2011': 10, '12_2012': 11, '12_2013': 12, '1_2010': 13, '1_2011': 14, '1_2012': 15, '1

In [3]:
# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print("Using device:", device)

Using device: cuda


In [4]:
# Embedding
embedding_dim = 16
# lstm hidden size
lstm_hidden = 128
# fully connected hidden
fc_hidden = 128
# dropout probability
p_dropout = 0.1

model = LSTMCollectiveIDP(activity_vocab_size=activity_ids_vocab_size_with_default,
                          resource_vocab_size=resource_ids_vocab_size_with_default,
                          month_vocab_size=month_ids_vocab_size_with_default,
                          num_trace_features=number_trace_attr,
                          num_output_labels=number_deviations_y,
                          # hyperparams provided in paper:
                          embedding_dim=embedding_dim,
                          lstm_hidden=lstm_hidden,
                          fc_hidden=fc_hidden,
                          dropout=p_dropout,
                          device=device)

/home/PSPLab/.local/share/virtualenvs/conformance_dev_and_pos_pred-Wn2oNUFZ/lib/python3.12/site-packages/torch/nn/init.py:566: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


In [5]:
# not mentioned in the paper:
# from code in paper
batch_size=128
# from code in paper
shuffle = True
epochs = 300 # 300 if early stopping (20% val from all train)
# from code in paper
learning_rate = 0.0001

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

optimizer_values = {"optimizer":optimizer,
                    "epochs":epochs,
                    "mini_batches":batch_size,
                    "shuffle": shuffle}

# Training with ealy stopping according to journal paper: patience:10, min_delta = 0
training = Training(model=model,
                    train_set=train_set,
                    val_set=val_set,
                    optimizer_values=optimizer_values,
                    loss_mode = 'collective',
                    device=device,
                    saving_path='./LSTM_collecctive_IDP.pkl')

history = training.train()

Epoch 1/300:   0%|          | 0/89 [00:00<?, ?it/s]

Epoch 1/300 — train loss: 0.2654, val loss: 0.1700


Epoch 2/300:   0%|          | 0/89 [00:00<?, ?it/s]

Epoch 2/300 — train loss: 0.1724, val loss: 0.1645


Epoch 3/300:   0%|          | 0/89 [00:00<?, ?it/s]

...

Epoch 119/300 — train loss: 0.1125, val loss: 0.1185


Epoch 120/300:   0%|          | 0/89 [00:00<?, ?it/s]

Epoch 120/300 — train loss: 0.1128, val loss: 0.1202


Epoch 121/300:   0%|          | 0/89 [00:00<?, ?it/s]

Epoch 121/300 — train loss: 0.1131, val loss: 0.1187
Early stopping triggered.
